In [1]:
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import h5py
import numpy as np
import scipy.io
from os import listdir
from os import mkdir
import pyarrow

# Indicando as pastas de fonte dos arquivos originais da região 5 e destino dos arquivos chaves

In [2]:
"""Pasta de origem dos dados e pasta de destino para os dados trabalhados"""
path_sources = r'C:\Users\vinic\python_projects\monitoracao_project\dados'
path_destiny = r'C:\Users\vinic\python_projects\monitoracao_project\dados_parquet'

folders = listdir(path_sources) # Lembrar de tirar o [0] quando for rodar tudo
folders = [folders[1]] # teste com uma pasta só
print(folders)

['LRV4306_accelerometer_data_teste']


# Código para criação do dataframe das chaves de acesso 

In [10]:
for i in range(len(folders)):

    if 'accel' in folders[i]:
        Chaves = pd.DataFrame(columns=['Data', 'N Passagens', 'Direcao', 'Sensor', 'Chave'])

    elif 'gps' in folders[i]:
        Chaves = pd.DataFrame(columns=['Data', 'N Passagens', 'Direcao', 'Chave'])
    
    name = folders[i]

    path_sour = path_sources+'\\'+folders[i] # entra na pasta com arquivos
    path_dest = path_destiny+'\\'+folders[i] # entra na pasta onde os arquivos serão salvos

    try:
        mkdir(path_dest) # cria pastas de acordo com o nome da pasta de origem
    except:
        pass
    
    files = listdir(path_sour) # Carrega os nomes dos arquivos dentro da pasta de origem

        # Quantidade de linhas no arquivo?


    for n in range(len(files)):
    
        j = files[n]

        props = j.split('_')
        props[-1] = props[-1][0]

        l = path_sour+'\\'+j
        m = path_dest+'\\'+j

        if 'accel' in folders[i]:
            #{file#}_{date}_{root index}_{daily passing#}_{region#}_{running direction}_{sensor channel}.mat
            df_temp = pd.DataFrame({'Data':[props[1]], 'N Passagens':[props[3]], 'Direcao':[props[5]], 'Sensor':[props[6]], 'Chave': l})
            Chaves = pd.concat([Chaves, df_temp], axis=0, ignore_index=True)


        elif 'gps' in folders[i]:

            df_temp = pd.DataFrame({'Data':[props[1]], 'N Passagens':[props[3]], 'Direcao':[props[5]], 'Chave':l})
            Chaves = pd.concat([Chaves, df_temp], axis=0, ignore_index=True)


    Chaves.to_parquet(path_dest+'\\'+name+'_key', engine = 'pyarrow', compression= 'brotli')

# Carregamento de arquivos específicos

In [3]:
# SERA EXCLUIDO DEPOIS

errors = [] # Guarda qualquer arquivo que falhou em ser carregado

for i in range(len(folders)):

    path_sour = path_destiny+'\\'+folders[i] # entra na pasta com as chaves
    
    files = listdir(path_sour) # Carrega os nomes dos arquivos dentro da pasta de origem

    for n in range(len(files)):
        if 'key' in files[i]:

            l = path_sour+'\\'+files[i]
            
            Chaves = pd.read_parquet(l)
            
            datas = set(Chaves.loc[:,'Data'])
            n_passagens = set(Chaves.loc[:,'N Passagens'])
            direcoes = set(Chaves.loc[:,'Direcao'])

            if 'accel' in files[n]:
                sensores = set(Chaves.loc[:,'Sensor'])
            elif 'gps' in files[n]:
                sensores = None
            


In [9]:
direcoes

{'1'}

## Teste sensor 1 dados

In [ ]:
# tratar dados que são None
print('Teste de qualidade dos dados')
print('--------------------------------')
for i in Chaves.keys():
    print(f'Elementos "None" na coluna {i}?')
    print(set(Chaves.loc[:, 'Sensor'] == None))
    print(f'Tamanho na coluna {i}')
    print(len(Chaves.loc[:, 'Sensor']))
    print('-----------------------------------')

In [82]:
errors = []
for i in range(len(folders)):

    path_sour = path_destiny+'\\'+folders[i] # entra na pasta com as chaves
    
    files = listdir(path_sour)

    for j in range(len(files)):

        if 'key' in files[i]: # carrega só os arquivos de chave

            l = path_sour+'\\'+files[i]
            
            Chaves = pd.read_parquet(l)
            
            datas = set(Chaves.loc[:,'Data'])
            n_passagens = set(Chaves.loc[:,'N Passagens'])
            direcoes = set(Chaves.loc[:,'Direcao'])

            if 'accel' in files[j]:
                sensores = set(Chaves.loc[:,'Sensor'])
            elif 'gps' in files[j]:
                sensores = None


        if 'accel' in folders[i]: 
            for p in datas:
                for q in n_passagens:
                    for r in direcoes:

                        df_temp = pd.DataFrame()
                        linhas_chave = (Chaves.loc[:,['Data', 'N Passagens', 'Direcao']]==[p, q, r]).all(axis=1)

                        for s in sensores:

                            for t in range(len(linhas_chave)):

                                if linhas_chave.iloc[t]:

                                    try:
                                        with h5py.File(l, 'r') as f:
                                            arq_carregado = np.asarray(f.get('save_var')).T                                    
                                    except:
                                        try:
                                            arq_carregado = scipy.io.loadmat(l)
                                            arq_carregado = np.asarray(arq_carregado[list(arq_carregado.keys())[-1]])
                                            
                                        except:
                                            errors.append(l) 

                                    df_temp[f'Sensor '+s] = arq_carregado[:,0]


        elif 'gps' in folders[i]:
            for p in datas:
                for q in n_passagens:
                    for r in direcoes:
                        df_temp = pd.DataFrame(columns = ['latitude', 'longitude', 'altura'])
        
                        linhas_chave = (Chaves.loc[:,['Data', 'N Passagens', 'Direcao']]==[p, q, r]).all(axis=1)

                        for t in range(len(linhas_chave)):

                                if linhas_chave.iloc[t]:

                                    try:
                                        with h5py.File(l, 'r') as f:
                                            arq_carregado = np.asarray(f.get('save_var')).T                                    
                                    except:
                                        try:
                                            arq_carregado = scipy.io.loadmat(l)
                                            arq_carregado = np.asarray(arq_carregado[list(arq_carregado.keys())[-1]])
                                            
                                        except:
                                            errors.append(l) 

                                    arq_carregado = pd.read_parquet(linhas_chave[-1]) 

                                    df_temp['Data'] = arq_carregado[:,0]

                                    df_temp['N passagens'] = arq_carregado[:,1]

                                    df_temp['Direcao'] = arq_carregado[:,2]


In [83]:
df_temp

,Sensor 2,Sensor 3,Sensor 1
0,0.313770,0.313770,0.313770
1,0.123914,0.123914,0.123914
2,0.009619,0.009619,0.009619
3,0.170537,0.170537,0.170537
4,0.132229,0.132229,0.132229
...,...,...,...
865442,0.172725,0.172725,0.172725
865443,0.127627,0.127627,0.127627
865444,0.201840,0.201840,0.201840
865445,0.342721,0.342721,0.342721


In [43]:
Chaves.iloc[t,-1]

'C:\\Users\\vinic\\python_projects\\monitoracao_project\\dados\\LRV4306_accelerometer_data_teste\\37_20131121_1_1_5_1_1.mat'

#Transformacao de coordenada em distância